In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
data0 = pd.read_csv("../input/covid19-indonesia/covid_19_indonesia_time_series_all.csv")
data0[-5:]

In [ ]:
data0['Location'].unique()

In [ ]:
import datetime
from datetime import date
date2=[]
for item in data0['Date']:
    item2=item.split('/')
    month=int(item2[0])
    day=int(item2[1])
    year=int(item2[2])
    date2+=[datetime.date(year,month,day)]
data0['Date2']=date2
data0

In [ ]:
data1=data0.groupby('Date2',as_index=False).sum()
data1

In [ ]:
data2=data1[['Date2','New Cases']]
data2

In [ ]:
data2['ratio to 7 days before']=1

In [ ]:
data2

In [ ]:
n=len(data2)
for i in range(n):    
    cpi=data2['New Cases'][i]
    data2.loc[i+7,'positives 7 days before']=cpi
data2[100:114]

In [ ]:
data2['ratio to 7 days before']=data2['New Cases']*100/data2['positives 7 days before']
data2['mean ratio to 7 days before']=data2['ratio to 7 days before'].rolling(window=7).mean()
data3=data2[431:-7]
data3

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data3['Date2'],y=data3['ratio to 7 days before'],name='ratio to 7 days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['Date2'],y=data3['mean ratio to 7 days before'],name='mean ratio to 7 days before'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Ratio to 7 days before in Indonesia")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Ratio",secondary_y=False)
fig.show()

In [ ]:
data4=data3[-7:]
data4

In [ ]:
N=[]
for i in range(40):
    N+=[i]
NUMB=pd.DataFrame(N)

In [ ]:
data5=pd.concat([data4,NUMB],axis=0)
data5=data5.drop(0,axis=1).reset_index(drop=True)
data5[0:10]

In [ ]:
data5['pcr_positives150']=data5['New Cases']
data5['pcr_positives130']=data5['New Cases']
data5['pcr_positives90']=data5['New Cases']

In [ ]:
for i in range(7,47):
    data5.loc[i,'Date2']=data5.loc[i-1,'Date2']+datetime.timedelta(days=1)
    data5.loc[i,'pcr_positives150']=data5.loc[i-7,'pcr_positives150']*1.5
    data5.loc[i,'pcr_positives130']=data5.loc[i-7,'pcr_positives130']*1.3    
    data5.loc[i,'pcr_positives90']=data5.loc[i-7,'pcr_positives90']*0.9
data6=data5[['Date2','New Cases','pcr_positives150','pcr_positives130','pcr_positives90']]
data6

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives150'],name='150% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives130'],name='130% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['pcr_positives90'],name='90% incresase to 7-days before'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data6['Date2'],y=data6['New Cases'],name='actual daily new cases'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Prediction of daily new cases in Indonesia as of 2021/07/09")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number",secondary_y=False)
fig.show()